In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import style
style.use('ggplot')
import itertools

from scipy.stats import gamma
from scipy.optimize import minimize_scalar

In [ ]:
# likelihood
alpha = 5 # known
beta  = 1 # unknown

# data
N = 10
x = gamma.rvs(a=alpha, scale=1/beta, size=N)

In [ ]:
# Plot dataset and "true" likelihood, just to get an idea of the dataset
xplot = np.linspace(1e-6,16,1e3)
fig = plt.figure(figsize=(15,5))
ax = plt.subplot()
ax.hist(x,bins=50, normed=True)
ax.plot(xplot, gamma.pdf(xplot,a=alpha,scale=1.0/beta) )
gif = plt.plot

In [ ]:
# Print statistics of data
print 'x.min:\t',
print x.min()
print 'x.mean:\t',
print x.mean()
print 'x.var:\t',
print x.var()
print 'x.max:\t',
print x.max()

In [ ]:
# prior on beta
alpha0 = 1
beta0  = 1

# posterior on beta
alphaN = alpha0 + N*alpha
betaN  = beta0  + np.sum(x)

# samples from posterior
post_samples = gamma.rvs(a=alphaN,scale=1.0/betaN, size=int(1e5))

# target beta value
xtarget = 15
target_log_pred = np.log(np.mean(gamma.pdf(xtarget,a=alpha,scale=1.0/post_samples)))

# form scalar loss function
def g(xstar):
    a = np.log(np.mean(gamma.pdf(xstar,a=alpha,scale=1.0/post_samples))) 
    return (a-target_log_pred)**2
res = minimize_scalar(g, bounds=(0, 1), method='bounded')
xstarres = res.x

print 'target x:\t',
print xtarget
print 'found x:\t',
print xstarres
print

print 'SANITY CHECK: these numbers should be close!'
print np.log(np.mean(gamma.pdf(xstarres,a=alpha,scale=1.0/post_samples)))
print target_log_pred

In [ ]:
xplot = np.linspace(1e-6,2.0,2e2)

fig = plt.figure(figsize=(15,5))
ax = plt.subplot()
colors = itertools.cycle(plt.rcParams['axes.prop_cycle'])

thiscolor = next(colors)
ax.plot(xplot, gamma.pdf(xplot,a=alphaN,scale=1.0/betaN), 
        color=thiscolor['color'], ls=':', lw=2, label='posterior')
thiscolor = next(colors)
ax.plot(xplot, 
        gamma.pdf(xstarres,a=alpha,scale=1.0/xplot), 
        color=thiscolor['color'], lw=2, label='likelihood of xstarres')
ax.plot(xplot, 
        gamma.pdf(xplot,a=alphaN,scale=1.0/betaN)
        *
        gamma.pdf(xstarres,a=alpha,scale=1.0/xplot), 
        color=thiscolor['color'], lw=2, 
        alpha=0.5, label='int likelihood of xstarres')
thiscolor = next(colors)
ax.plot(xplot, 
        gamma.pdf(xtarget,a=alpha,scale=1.0/xplot), 
        color=thiscolor['color'], lw=2, label='likelihood of xtarget')
ax.plot(xplot, 
        gamma.pdf(xplot,a=alphaN,scale=1.0/betaN)
        *
        gamma.pdf(xtarget,a=alpha,scale=1.0/xplot), 
        color=thiscolor['color'], lw=2, 
        alpha=0.5, label='int likelihood of xtarget')

ax.set_xlim([0,2])
ax.set_ylim([0,0.06])
ax.legend(loc=1)
gif = plt.plot

In [ ]:
print 'log Bayesian predictive of found x\t:',
print np.log(np.mean(gamma.pdf(xstarres,a=alpha,scale=1.0/post_samples)))
print 'log Bayesian predictive of target x\t:',
print np.log(np.mean(gamma.pdf(xtarget,a=alpha,scale=1.0/post_samples)))
print
print 'WAPDI of found x\t:',
print np.var(gamma.logpdf(xstarres,a=alpha,scale=1.0/post_samples)) \
      / np.log(np.mean(gamma.pdf(xstarres,a=alpha,scale=1.0/post_samples)))
print 'WAPDI of target x\t:',    
print np.var(gamma.logpdf(xtarget,a=alpha,scale=1.0/post_samples)) \
      / np.log(np.mean(gamma.pdf(xtarget,a=alpha,scale=1.0/post_samples)))